In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/VNAV/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1Xg2hxidUwcEigNrOyCh3l0l05EG9Xoxe/VNAV


In [ ]:
# Gardens point dataset
"""
    From: https://zenodo.org/record/4590133#.ZAdKktJBxH5
    960 x 540 dim
"""


# from utilities import CustomDataset

def path_to_pil_img(path):
    return Image.open(path).convert("RGB")

# base_transform = T.Compose([
#     T.ToTensor(),
#     T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# mixVPR_transform = T.Compose([
#     T.ToTensor(),
#     T.Normalize(mean=[0.485, 0.456, 0.406],
#                             std=[0.229, 0.224, 0.225]),
#     T.Resize((320,320))
# ])

class Gardens():
    """
    Returns dataset class with images from database and queries for the gardens dataset.
    """
    def __init__(self,datasets_folder='./gardens',dataset_name="gardens",split="train",use_ang_positives=False,dist_thresh = 10,ang_thresh=20,use_mixVPR=False,use_SAM=False):
        # super().__init__()

        self.dataset_name = dataset_name
        self.datasets_folder = datasets_folder
        self.split = split
        self.use_mixVPR = use_mixVPR
        self.use_SAM = use_SAM
        self.db_paths = natsort.natsorted(os.listdir(os.path.join(self.datasets_folder,self.dataset_name,"day_right")))
        self.q_paths = natsort.natsorted(os.listdir(os.path.join(self.datasets_folder,self.dataset_name,"day_left")))

        self.db_abs_paths = [] # day_right
        self.q_abs_paths_day_left = []
        self.q_abs_paths_night_right = []
        self.q_abs_paths = self.q_abs_paths_day_left + self.q_abs_paths_night_right

        # this sets day_right as database, night_right as query set
        # I also want to evaluate using day_left as query set. idk why they use day_left to set q_paths and its ignored later
        for p in self.db_paths:
            self.db_abs_paths.append(os.path.join(self.datasets_folder,self.dataset_name,"day_right",p))

        for q in self.q_paths:
            self.q_abs_paths_night_right.append(os.path.join(self.datasets_folder,self.dataset_name,"night_right",q))
            self.q_abs_paths_day_left.append(os.path.join(self.datasets_folder,self.dataset_name,"day_left",q))

        self.q_abs_paths = self.q_abs_paths_day_left + self.q_abs_paths_night_right


        self.db_num = len(self.db_abs_paths)
        self.q_num = len(self.q_abs_paths)
        self.q_num_night_right = len(self.q_abs_paths_night_right)
        self.q_num_day_left = len(self.q_abs_paths_day_left)


        self.database_num = self.db_num
        self.queries_num = self.q_num

        self.gt_positives = np.load(os.path.join(self.datasets_folder,self.dataset_name,"gardens_gt.npy"),allow_pickle=True) #returns dictionary of gardens dataset

        self.images_paths = list(self.db_abs_paths) + list(self.q_abs_paths)

        self.soft_positives_per_query = []

        for i in range(len(self.gt_positives)):
            self.soft_positives_per_query.append(self.gt_positives[i][1]) #corresponds to index wise soft positives

gardens_obj = Gardens(datasets_folder='./',
             dataset_name="gardens",
             split="train",
             use_ang_positives=False,
             dist_thresh = 10,
             ang_thresh=20,
             use_mixVPR=False,
             use_SAM=False
            )

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


# Reading the image and converting into B/W
image = cv2.imread('book.jpg')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


def computeFeatures(img):
    # Inputs
    # img: 3-D numpy array of an RGB color image
    #
    # Output
    # featvect: A D-dimensional vector of the input image 'img'

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sift = cv2.xfeatures2d.SIFT_create(contrastThreshold=0.0)
    kps, descriptors = sift.detectAndCompute(gray, None)

    # Applying the function
    kp_image = cv2.drawKeypoints(image, kp, None, color=(
        0, 255, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imshow('SIFT', kp_image)
    cv2.waitKey()


    return descriptors # featvect

In [22]:

import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm


In [10]:



# takes all images and convert them to grayscale.
# return a dictionary that holds all images category by category.
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        # category = []
        path = folder + "/" + filename
        img = cv2.imread(path)
        # print(img.shape)
        # for cat in os.listdir(path):
        #     img = cv2.imread(path + "/" + cat,0)
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # if img is not None:
            #     category.append(img)
        images.append(img)
    return images

In [13]:
reference_imgs = load_images_from_folder('./gardens/day_left')  # take all images category by category
day_right_imgs = load_images_from_folder("./gardens/day_right") # take test images


In [24]:
# Creates descriptors using sift
# Takes one parameter that is images dictionary
# Return an array whose first index holds the decriptor_list without an order
# And the second index holds the sift_vectors dictionary which holds the descriptors but this is seperated class by class
def sift_features(images):
    # sift_vectors = {}
    sift_vectors = []
    descriptor_list = []
    sift = cv2.xfeatures2d.SIFT_create()

    for img in tqdm(images):
        kp, des = sift.detectAndCompute(img,None)
        descriptor_list.extend(des)

    # for key,value in images.items():
    #     features = []
    #     for img in value:
    #         kp, des = sift.detectAndCompute(img,None)

    #         descriptor_list.extend(des)
    #         features.append(des)
    #     sift_vectors[key] = features
    return descriptor_list#, sift_vectors

# unordered descriptor list and sift features for reference data
descriptor_list = sift_features(reference_imgs)
# descriptor_list, all_bovw_feature = sift_features(reference_imgs)
# Takes the sift features that is seperated class by class for test data
# test_bovw_feature = sift_features(day_right_imgs)[1]  # only used for hist viewing

  0%|          | 0/200 [00:00<?, ?it/s]

In [35]:
len(descriptor_list)

410601

In [25]:
# A k-means clustering algorithm who takes 2 parameter which is number
# of cluster(k) and the other is descriptors list(unordered 1d array)
# Returns an array that holds central points.
def kmeans(k, descriptor_list):
    kmeans = KMeans(n_clusters = k, n_init=10)
    kmeans.fit(descriptor_list)
    visual_words = kmeans.cluster_centers_
    return visual_words

# Takes the central points which is visual words
visual_words = kmeans(32, descriptor_list)

In [29]:
np.save(f"./gardens/cache/kmeans_32_bow.npy", visual_words)

In [ ]:
# # Takes 2 parameters. The first one is a dictionary that holds the descriptors that are separated class by class
# # And the second parameter is an array that holds the central points (visual words) of the k means clustering
# # Returns a dictionary that holds the histograms for each images that are separated class by class.
# def image_class(all_bovw, centers):
#     dict_feature = {}
#     for key,value in all_bovw.items():
#         category = []
#         for img in value:
#             histogram = np.zeros(len(centers))
#             for each_feature in img:
#                 ind = find_index(each_feature, centers)
#                 histogram[ind] += 1
#             category.append(histogram)
#         dict_feature[key] = category
#     return dict_feature

# # Creates histograms for train data
# bovw_train = image_class(all_bovw_feature, visual_words)
# # Creates histograms for test data
# bovw_test = image_class(test_bovw_feature, visual_words)

In [30]:
# run knn function for querying
# def knn(k, database, query):
k = 5
neigh = NearestNeighbors(n_neighbors=k)
neigh.fit(visual_words)

gt_positives = np.load(("./gardens/gardens_gt.npy"),allow_pickle=True) #returns dictionary of gardens dataset
soft_positives_per_query = []

for i in range(len(gt_positives)):
  soft_positives_per_query.append(gt_positives[i][1]) #corresponds to index wise soft positives


# results_bowl = knn(bovw_train, bovw_test)

In [33]:
visual_words.shape

(32, 128)

In [32]:

query_index = 0
qu = day_right_imgs[query_index]
print(qu)
distances, indices = neigh.kneighbors(qu)

[[[144 130 112]
  [144 130 112]
  [144 130 112]
  ...
  [ 38  40  41]
  [ 40  42  43]
  [ 43  45  46]]

 [[144 130 112]
  [144 130 112]
  [144 130 112]
  ...
  [ 38  40  41]
  [ 40  42  43]
  [ 43  45  46]]

 [[144 130 112]
  [144 130 112]
  [144 130 112]
  ...
  [ 38  40  41]
  [ 40  42  43]
  [ 43  45  46]]

 ...

 [[ 88  78  71]
  [ 88  78  71]
  [ 87  77  70]
  ...
  [ 20  21  31]
  [ 18  20  30]
  [ 19  21  31]]

 [[ 89  77  71]
  [ 89  77  71]
  [ 88  76  70]
  ...
  [ 20  21  31]
  [ 18  20  30]
  [ 19  21  31]]

 [[ 89  77  71]
  [ 89  77  71]
  [ 88  76  70]
  ...
  [ 20  21  31]
  [ 18  20  30]
  [ 19  21  31]]]


ValueError: ignored